### Notes

The models you can run here will be limited by what's implemented by the litellm library, because that's what managing the API calls.

You can see the models here: https://docs.litellm.ai/docs/



In [1]:
from functions import *


C:\Users\abiga\OneDrive\Documents\PythonScripts\llm_eval\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Parameters

In [4]:
models_dict = {
    #'claude-2.1':  "ANTHROPIC", 
    'gpt-3.5-turbo-0301': "OPENAI"
               } 
csv_file_path = '../data/prompt_target_answer_pairs.csv'
similarity_model_name = 'sentence-transformers/paraphrase-mpnet-base-v2'
num_runs = 2
temperature = .9
num_perturbations= 1
is_file_path = True
llm_evaluation_model = ['gpt-4', "OPENAI"]
instructions = "Please answer thoroughly: "
perturbation_model = ['gpt-3.5-turbo-0301', "OPENAI"]

pipeline = LLMAnalysisPipeline(
    input_data=csv_file_path, 
    models_dict=models_dict, 
    perturbation_model=perturbation_model, 
    llm_evaluation_model=llm_evaluation_model,
    temperature = temperature,
    num_runs= num_runs,
    is_file_path = is_file_path,
    similarity_model_name = similarity_model_name,
    num_perturbations = num_perturbations,
    instructions = instructions

)

# Run the pipeline
df_responses = pipeline.run_pipeline()




2023-12-13 15:50:35,289 - INFO - Reading prompts from CSV file.
2023-12-13 15:50:35,925 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-12-13 15:50:36,670 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-12-13 15:50:37,248 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-12-13 15:50:38,251 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-12-13 15:50:39,133 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-12-13 15:50:40,316 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-12-13 15:51:00,636 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-12-13 15:51:21,751 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-12-13 15:51:41,189 - INFO -

In [10]:
df_responses.groupby(['model', 'original_prompt']).count()

response  \
model              original_prompt                                                
claude-2.1         How do astroflora perform photosynthesis in low...         2   
                   What do genes consist of and how do they work?             1   
                   What is the basic unit of life and what does th...         1   
                   What is the powerhouse of the cell and how does...         1   
                   What particle stabilizes the membrane of hover ...         1   
                   Which organelle contains the cell's futurogenic...         2   
gpt-3.5-turbo-0301 How do astroflora perform photosynthesis in low...         2   
                   What do genes consist of and how do they work?             1   
                   What is the basic unit of life and what does th...         1   
                   What is the powerhouse of the cell and how does...         1   
                   What particle stabilizes the membrane of hover ...         1   
                   Which organelle contains the cell's futurogenic...         2   

                                                                       temperature  \
model              original_prompt                                                   
claude-2.1         How do astroflora perform photosynthesis in low...            2   
                   What do genes consist of and how do they work?                1   
                   What is the basic unit of life and what does th...            1   
                   What is the powerhouse of the cell and how does...            1   
                   What particle stabilizes the membrane of hover ...            1   
                   Which organelle contains the cell's futurogenic...            2   
gpt-3.5-turbo-0301 How do astroflora perform photosynthesis in low...            2   
                   What do genes consist of and how do they work?                1   
                   What is the basic unit of life and what does th...            1   
                   What is the powerhouse of the cell and how does...            1   
                   What particle stabilizes the membrane of hover ...            1   
                   Which organelle contains the cell's futurogenic...            2   

                                                                       actual_prompt  \
model              original_prompt                                                     
claude-2.1         How do astroflora perform photosynthesis in low...              2   
                   What do genes consist of and how do they work?                  1   
                   What is the basic unit of life and what does th...              1   
                   What is the powerhouse of the cell and how does...              1   
                   What particle stabilizes the membrane of hover ...              1   
                   Which organelle contains the cell's futurogenic...              2   
gpt-3.5-turbo-0301 How do astroflora perform photosynthesis in low...              2   
                   What do genes consist of and how do they work?                  1   
                   What is the basic unit of life and what does th...              1   
                   What is the powerhouse of the cell and how does...              1   
                   What particle stabilizes the membrane of hover ...              1   
                   Which organelle contains the cell's futurogenic...              2   

                                                                       target_answer  \
model              original_prompt                                                     
claude-2.1         How do astroflora perform photosynthesis in low...              2   
                   What do genes consist of and how do they work?                  1   
                   What is the basic unit of life and what does th...              1   
                   What is the

In [7]:
def aggregate_best_scores(df, score_column):
    # Define the columns to include in the output
    relevant_columns = ['model', 'original_prompt', 'actual_prompt', 'response', score_column]

    # Group by 'model' and 'original_prompt', and get the row with the best score in each group
    df_best_scores = df.loc[df.groupby(['model', 'original_prompt'])[score_column].idxmax()]

    # Select only relevant columns for output
    df_best_scores = df_best_scores[relevant_columns]

    return df_best_scores


sample=aggregate_best_scores(df_responses, "similarity_scores")
len(sample)

12